In [1]:
# Imports
%gui tk
import tkinter as tk
import math
import time
import pandas as pd
from collections import defaultdict
import random
import heapq

In [5]:
# Piece classes
class Fox:
    def __init__(self, board, row, col, radius_frac=0.35, color="red"):
        self.board = board
        self.row = row
        self.col = col
        self.color = color
        self.radius_frac = radius_frac
        self.item_id = self.board._draw_piece(self, color=self.color, radius_frac=self.radius_frac, owner="Fox")
        self.board._occupy(self, row, col)

    def move(self, direction): # direction = Up, Down, Left, or Right
        if not self.board._is_turn(self) or self.board.game_over:
            return False
        dr, dc = self.board._dir_delta(direction)
        if dr is None:
            return False
        nr, nc = self.row + dr, self.col + dc
        if not self.board._in_bounds(nr, nc) or not self.board._is_empty(nr, nc):
            return False
        self.board._relocate(self, nr, nc)
        if self.board._fox_on_far_side():
            self.board._end_game(winner="Fox")
            return True
        self.board._next_turn()
        return True


class Hound:
    def __init__(self, board, row, col, radius_frac=0.35, color="blue"):
        self.board = board
        self.row = row
        self.col = col
        self.color = color
        self.radius_frac = radius_frac
        self.item_id = self.board._draw_piece(self, color=self.color, radius_frac=self.radius_frac, owner="Hounds")
        self.board._occupy(self, row, col)

    def move(self, direction): # direction = Up, Down, Left, or Right
        if not self.board._is_turn(self) or self.board.game_over:
            return False
        if direction.lower() == "down":  # Hounds cannot move down
            return False
        dr, dc = self.board._dir_delta(direction)
        if dr is None:
            return False
        nr, nc = self.row + dr, self.col + dc
        if not self.board._in_bounds(nr, nc) or not self.board._is_empty(nr, nc):
            return False
        self.board._relocate(self, nr, nc)
        if not self.board._fox_has_moves():
            self.board._end_game(winner="Hounds")
            return True
        self.board._next_turn()
        return True


# Main Gameboard
class gameboard:
    MODES = ("Player", "FoxRandom", "HoundsRandom", "ShortestPath", "Mini-Max")

    def __init__(self, master, rows=6, cols=6, tile_size=60,
                 dark="#555555", light="#DDDDDD", outline="black"):
        self.master = master
        self.rows = rows
        self.cols = cols
        self.tile_size = tile_size
        self.dark = dark
        self.light = light
        self.outline = outline

        # geometry
        self.r = tile_size / math.sqrt(2)
        self.pad_x = self.r + 2
        self.pad_y = self.r + 2

        # canvas size
        self.width  = (cols + rows) * self.r + 2 * self.pad_x
        self.height = (cols + rows) * self.r + 2 * self.pad_y

        # UI container
        self.top = tk.Frame(master)
        self.top.pack()

        # Canvas
        self.canvas = tk.Canvas(self.top, width=self.width, height=self.height,
                                bg="white", highlightthickness=0)
        self.canvas.pack(pady=5)

        # Game state
        self.occ = {}
        self.item_to_piece = {}
        self.selected = None
        self.current_turn = "Fox"
        self.game_over = False
        self.winner = None
        self.fox_mode = "Player"
        self.hounds_mode = "Player"
        self.fox = None
        self.hounds = []

        self.canvas.bind("<Button-1>", self._on_click_canvas)
        self.master.bind("<Key>", self._on_keypress)

        self._draw_board()
        self._create_turn_hud("Waiting…")

    # Start and end games
    def game_start(self, fox_mode="Player", hounds_mode="Player"):
        self.fox_mode, self.hounds_mode = fox_mode, hounds_mode
        self._start_new_game()
        return self.winner  # returns after game ends (if blocking)

    def quit(self):
        try:
            self.master.destroy()
        except Exception:
            pass

    # Setup
    def _start_new_game(self):
        # Clear board
        self.canvas.delete("piece")
        self.canvas.delete("overlay")
        self.occ.clear()
        self.item_to_piece.clear()
        self.selected = None
        self.current_turn = "Fox"
        self.game_over = False
        self.winner = None

        # Create pieces
        self.fox = Fox(self, 0, 0)
        self.hounds = [
            Hound(self, 5, 5),
            Hound(self, 5, 3),
            Hound(self, 4, 4),
            Hound(self, 3, 5)
        ]

        self.canvas.tag_raise(self.fox.item_id)
        for h in self.hounds:
            self.canvas.tag_raise(h.item_id)

        self._update_turn_hud(self.current_turn)

        start_mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if start_mode == "FoxRandom":
            self.Fox_Random_AI()
        elif start_mode == "HoundsRandom":
            self.Hounds_Random_AI()
        elif start_mode == "ShortestPath":
            self.Fox_Short_Path_AI()
        elif start_mode == "Mini-Max":
            self.Hounds_Minimax_AI(0, -float("inf"), float("inf"))

    # Drawing
    def _draw_board(self):
        self.canvas.delete("tile")
        for r in range(self.rows):
            for c in range(self.cols):
                cx, cy = self._tile_center(r, c)
                coords = [cx, cy - self.r, cx + self.r, cy, cx, cy + self.r, cx - self.r, cy]
                color = self.dark if (r + c) % 2 == 0 else self.light
                self.canvas.create_polygon(coords, fill=color, outline=self.outline, tags=("tile",))

    def _create_turn_hud(self, turn_text):
        self.canvas.delete("turnhud")
        pad = 10
        w, h = 120, 30
        self.canvas.create_rectangle(pad, pad, pad + w, pad + h,
                                     fill="#f4f4f4", outline="#555", tags=("turnhud",))
        self.canvas.create_text(pad + w / 2, pad + h / 2,
                                text=f"Turn: {turn_text}", font=("Arial", 10, "bold"),
                                tags=("turnhud",))

    def _update_turn_hud(self, turn_text):
        self._create_turn_hud(turn_text)

    # Geometry
    def _tile_center(self, r, c):
        return ((c - r) * self.r + (self.cols * self.r) + self.pad_x,
                (c + r) * self.r + self.pad_y)

    def _in_bounds(self, r, c):
        return 0 <= r < self.rows and 0 <= c < self.cols

    def _dir_delta(self, direction):
        d = direction.lower()
        return {
            "up": (-1, -1),
            "down": (1, 1),
            "left": (1, -1),
            "right": (-1, 1)
        }.get(d, (None, None))

    # Occupancy
    def _is_empty(self, r, c):
        return (r, c) not in self.occ

    def _occupy(self, piece, r, c):
        self.occ[(r, c)] = piece

    def _vacate(self, r, c):
        self.occ.pop((r, c), None)

    def _relocate(self, piece, nr, nc):
        self._vacate(piece.row, piece.col)
        piece.row, piece.col = nr, nc
        self._occupy(piece, nr, nc)
        self._position_piece(piece)

    # Turn conditions
    def _is_turn(self, piece):
        return (self.current_turn == "Fox" and isinstance(piece, Fox)) or \
               (self.current_turn == "Hounds" and isinstance(piece, Hound))

    def _next_turn(self):
        if self.game_over:
            return

        self.current_turn = "Hounds" if self.current_turn == "Fox" else "Fox"
        self._update_turn_hud(self.current_turn)

        # If Fox's turn begins and has no moves, Hounds instantly win.
        if self.current_turn == "Fox" and not self._fox_has_moves() and not self.game_over:
            self._end_game("Hounds")
            return

        mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if mode == "HoundsRandom":
            self.Hounds_Random_AI()
        elif mode == "FoxRandom":
            self.Fox_Random_AI()
        elif mode == "Mini-Max":
            self.Hounds_Minimax_AI(0, -float("inf"), float("inf"))
        elif mode == "ShortestPath":
            self.Fox_Short_Path_AI()

    def _fox_has_moves(self):
        r, c = self.fox.row, self.fox.col
        for d in ("Up", "Down", "Left", "Right"):
            dr, dc = self._dir_delta(d)
            nr, nc = r + dr, c + dc
            if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                return True
        return False

    def _fox_on_far_side(self):
        return (self.fox.row + self.fox.col) == (self.rows + self.cols - 2)

    def _end_game(self, winner):
        self.game_over = True
        self.winner = winner
        self._dim_and_announce(winner)

    # Overlay
    def _dim_and_announce(self, winner):
        self.canvas.create_rectangle(0, 0, self.width, self.height,
                                     fill="black", stipple="gray50", outline="", tags=("overlay",))
        text, color = ("Win", "green") if winner == "Fox" else ("Lose", "red")
        self.canvas.create_text(self.width / 2, self.height / 2,
                                text=text, fill=color,
                                font=("Arial", int(self.tile_size * 1.2), "bold"),
                                tags=("overlay",))
        self.canvas.tag_raise("overlay")

    # Piece Draw/Move
    def _draw_piece(self, piece, color, radius_frac=0.35, outline="white", owner=""):
        cx, cy = self._tile_center(piece.row, piece.col)
        r = self.tile_size * radius_frac
        item = self.canvas.create_oval(cx - r, cy - r, cx + r, cy + r,
                                       fill=color, outline=outline, width=2, tags=("piece", owner))
        self.item_to_piece[item] = piece
        return item

    def _position_piece(self, piece):
        cx, cy = self._tile_center(piece.row, piece.col)
        r = self.tile_size * piece.radius_frac
        self.canvas.coords(piece.item_id, cx - r, cy - r, cx + r, cy + r)

    # Manual play: click + arrow
    def _on_click_canvas(self, event):
        if self.game_over:
            return
        mode = self.fox_mode if self.current_turn == "Fox" else self.hounds_mode
        if mode != "Player":
            return
        items = self.canvas.find_overlapping(event.x, event.y, event.x, event.y)
        for item in reversed(items):
            if item in self.item_to_piece:
                piece = self.item_to_piece[item]
                if self._is_turn(piece):
                    self._highlight_selection(piece)
                    break

    def _highlight_selection(self, piece):
        if self.selected:
            self.canvas.itemconfigure(self.selected.item_id, width=2)
        self.selected = piece
        if piece:
            self.canvas.itemconfigure(piece.item_id, width=4)

    def _on_keypress(self, event):
        if self.game_over or not self.selected:
            return
        key_map = {"Up": "Up", "Down": "Down", "Left": "Left", "Right": "Right"}
        if event.keysym not in key_map:
            return
        moved = self.selected.move(key_map[event.keysym])
        if moved:
            self.canvas.itemconfigure(self.selected.item_id, width=2)
            self.selected = None

    # AI Movement
    def Fox_Random_AI(self):
        """
        Pick a random legal move for whichever side’s turn it is (Fox or Hounds).
        If no moves are available, end the game accordingly.
        """
        if self.game_over:
            return

        # Pick the active side
        if self.current_turn == "Fox":
            movers = [self.fox]
        else:
            movers = self.hounds

        legal_moves = [] # (piece, direction)

        for piece in movers:
            # candidate directions that preserve dark tiles
            dirs = ["Up", "Down", "Left", "Right"]

            for d in dirs:

                dr, dc = self._dir_delta(d)
                if dr is None:
                    continue

                nr, nc = piece.row + dr, piece.col + dc
                if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                    legal_moves.append((piece, d))

        if not legal_moves:
            # no moves available means losing condition for the current side
            loser = self.current_turn
            self._end_game(winner="Hounds" if loser == "Fox" else "Fox")
            return

        # Choose and execute a random move
        piece, direction = random.choice(legal_moves)
        piece.move(direction)



    def Hounds_Random_AI(self):
        """
        Pick a random legal move for whichever side’s turn it is (Fox or Hounds).
        If no moves are available, end the game accordingly.
        """
        if self.game_over:
            return

        # Pick the active side
        if self.current_turn == "Fox":
            movers = [self.fox]
        else:
            movers = self.hounds

        legal_moves = [] # (piece, direction)

        for piece in movers:
            # candidate directions that preserve dark tiles
            dirs = ["Up", "Down", "Left", "Right"]

            for d in dirs:
                # Hounds cannot move Down
                if isinstance(piece, Hound) and d == "Down":
                    continue

                dr, dc = self._dir_delta(d)
                if dr is None:
                    continue

                nr, nc = piece.row + dr, piece.col + dc
                if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                    legal_moves.append((piece, d))

        if not legal_moves:
            # no moves available means losing condition for the current side
            loser = self.current_turn
            self._end_game(winner="Hounds" if loser == "Fox" else "Fox")
            return

        # Choose and execute a random move
        piece, direction = random.choice(legal_moves)
        piece.move(direction)



    # Helper Functions for Minimax

    # Check if location is empty or has a fox in it
    def _is_empty_or_fox(self, r, c):
        return (r, c) not in self.occ or (self.fox.row == r and self.fox.col == c)

    # Relocate pieces without triggering next turn
    def _relocate_no_turn(self, piece, nr, nc):
        self._vacate(piece.row, piece.col)
        piece.row, piece.col = nr, nc
        self._occupy(piece, nr, nc)
        self._position_piece(piece)

    # Check if there is a winner on the current board, if fox is at the end or if the fox has no moves
    def check_winner(self):
        winner = None
        if self.fox.row == 5 and self.fox.col == 5:
            winner = "Fox"
        if not self._fox_has_moves() and not (self.fox.row == 5 and self.fox.col == 5):
            winner = "Hounds"
        return winner

    # Check if all of the hounds are above their starting positions
    def first_line_complete(self):
        if self._is_empty(5,3) and self._is_empty(4,4) and self._is_empty(3,5) and self._is_empty(5,5):
            return True
        else:
            return False

    # Check if all of the hounds are above their starting positions and above the line above those positions 
    def second_line_complete(self):
        if self.first_line_complete() and self._is_empty(5,1) and self._is_empty(4,2) and self._is_empty(3,3) and self._is_empty(2,4) and self._is_empty(1,5):
            return True
        else:
            return False

    # Check the board and assign points based on board state
    def check_board(self):
        score = 0

        # A first line state is 4 in a row on the line above starting positions - it blocks the fox from getting past while still moving up and should be priority 1
        left_first_row_line = not self._is_empty_or_fox(5,1) and not self._is_empty_or_fox(4,2) and not self._is_empty_or_fox(3,3) and not self._is_empty_or_fox(2,4)
        right_first_row_line = not self._is_empty_or_fox(4,2) and not self._is_empty_or_fox(3,3) and not self._is_empty_or_fox(2,4) and not self._is_empty_or_fox(1,5)

        first_line_state = left_first_row_line or right_first_row_line

        # A second line state is 4 in a row on the second line above starting positions with fox above it - it blocks the fox from getting past while still moving up and should be priority 2 depending on depth, if recursion runs deep enough a win may have already been found and is prioritized instead
        left_second_row_line = not self._is_empty_or_fox(4,0) and not self._is_empty_or_fox(3,1) and not self._is_empty_or_fox(2,2) and not self._is_empty_or_fox(1,3)
        right_second_row_line = not self._is_empty_or_fox(3,1) and not self._is_empty_or_fox(2,2) and not self._is_empty_or_fox(1,3) and not self._is_empty_or_fox(0,4)
        fox_position = False
        if self.fox.row == 0 and self.fox.col == 0:
            fox_position = True
        elif self.fox.row == 1 and self.fox.col == 1:
            fox_position = True
        elif self.fox.row == 0 and self.fox.col == 2:
            fox_position = True
        elif self.fox.row == 2 and self.fox.col == 0:
            fox_position = True

        second_line_state = (left_second_row_line or right_second_row_line) and fox_position

        # Flags for Completing
        first_line_complete_flag = self.first_line_complete()
        second_line_complete_flag = self.second_line_complete()
        
        # Points assigned for states 
        # First and second line states are priority 1 and 2 unless check_winner found a winner which will skip this and give 1000 points
        if first_line_state:
            score += 40
        if second_line_state:
            score += 80
        
        # Moving into the 1st row gives 3 points unless it is a piece already in the 1st row moving to the side, then it gives 2
        if not self._is_empty_or_fox(5,1):
            score += 2
        if not self._is_empty_or_fox(4,2):
            score += 3
        if not self._is_empty_or_fox(3,3):
            score += 3
        if not self._is_empty_or_fox(2,4):
            score += 3
        if not self._is_empty_or_fox(1,5):
            score += 2
        # Moving into the 2nd row only gives 2 points if the first_line_state hasn't been completed to keep individual pieces from running up before the others, if the first_line_state has been completed then it gives 42 to encourage movement past the state rather than returning to it
        if not self._is_empty_or_fox(4,0):
            if first_line_complete_flag:
                score += 42
            else:
                score += 2
        if not self._is_empty_or_fox(3,1):
            if first_line_complete_flag:
                score += 42
            else:
                score += 2
        if not self._is_empty_or_fox(2,2):
            if first_line_complete_flag:
                score += 42
            else:
                score += 2
        if not self._is_empty_or_fox(1,3):
            if first_line_complete_flag:
                score += 42
            else:
                score += 2
        # (Note: A win has probably been found before this movement) Moving into the 3nd row only gives 1 point if the second_line_state hasn't been completed to keep individual pieces from running up before the others, if the second_line_state has been completed then it gives 81 to encourage movement past the state rather than returning to it
        if not self._is_empty_or_fox(2,0):
            if second_line_complete_flag:
                score += 81
            else:
                score += 1
        if not self._is_empty_or_fox(1,1):
            if second_line_complete_flag:
                score += 81
            else:
                score += 1
        if not self._is_empty_or_fox(0,2):
            if second_line_complete_flag:

                score += 81
            else:
                score += 1
        return score
            
    
    # Score-Getting Minimax Function - Returns Score to provide main function with appropriate Move
    def minimax(self, depth, isMaximizing, alpha, beta, max_depth=4):

        # Terminating Condition = If fox has no moves (Hounds Win), Fox is at end (Fox Wins)
        if not self._fox_has_moves() or (self.fox.row == 5 and self.fox.col == 5) or depth == max_depth:

            # Check winner, return 1000 points +/- (100 * depth) to prioritize lower movement wins (aka: win in 1 turn instead of 3)
            winner = self.check_winner()
            if winner == "Hounds":
                return float(1000 - (100 * depth))
            if winner == "Fox":
                return float(-1000 + (100 * depth))
            else: 
                score = self.check_board()
                return score

        # Hounds Turn - Maximize the Score
        if isMaximizing == True:

            # Set best score to -infinity
            best_score = float('-inf')
            movers = self.hounds

            # Check legal moves for hounds and put in legal_moves array (moves include [piece, direction])
            legal_moves = []
    
            for piece in movers:
                # candidate directions that preserve dark tiles
                dirs = ["Up", "Down", "Left", "Right"]
    
                for d in dirs:
                    # Hounds cannot move Down
                    if isinstance(piece, Hound) and d == "Down":
                        continue
    
                    dr, dc = self._dir_delta(d)
                    if dr is None:
                        continue
    
                    nr, nc = piece.row + dr, piece.col + dc
                    if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                        legal_moves.append((piece, d))

            # For each legal move, check if it has the best score
            for move in legal_moves:
                # Get the piece and direction of the move
                piece, direction = move
                
                # Save the initial row and column position of the piece to return it to its original location, set up new variables
                old_row = piece.row
                old_column = piece.col
                new_row = 0
                new_column = 0

                # Use directions to create new_row and new_column for piece to move to
                if direction.lower() == "down":
                    new_column = piece.col + 1
                    new_row = piece.row + 1
                if direction.lower() == "up":
                    new_column = piece.col - 1
                    new_row = piece.row - 1
                if direction.lower() == "left":
                    new_column = piece.col - 1 
                    new_row = piece.row + 1
                if direction.lower() == "right":
                    new_column = piece.col + 1
                    new_row = piece.row - 1

                # Relocate the piece to the new position without triggering next turn
                piece.board._relocate_no_turn(piece, new_row, new_column)

                # Recursively call minimax to get the score
                score = self.minimax(depth + 1, False, alpha, beta, max_depth)

                # Relocate the piece back to its original location without triggering next turn
                piece.board._relocate_no_turn(piece, old_row, old_column)

                # Check if this score is greater than the best_score, if so replace the old best_score
                best_score = max(score, best_score)

                # Break if Alpha is >= Beta 
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break

            # If there are no legal moves, return 0
            if not legal_moves:
                return 0

            return best_score
        
        # Fox's turn
        else:

            # Set best score to infinity
            best_score = float('inf')
            moves = None
            fox_moved = None
            movers = [self.fox]
            
            # Check legal moves for fox and put in legal_moves array (moves include [piece, direction])
            legal_moves = []
    
            for piece in movers:
                # candidate directions that preserve dark tiles
                dirs = ["Up", "Down", "Left", "Right"]
    
                for d in dirs:
    
                    dr, dc = self._dir_delta(d)
                    if dr is None:
                        continue
    
                    nr, nc = piece.row + dr, piece.col + dc
                    if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                        legal_moves.append((piece, d))

            # For each legal move, check if it has the best score            
            for move in legal_moves:
                # Get the piece and direction of the move
                piece, direction = move

                # Save the initial row and column position of the piece to return it to its original location, set up new variables
                old_row = piece.row
                old_column = piece.col
                new_row = 0
                new_column = 0

                # Use directions to create new_row and new_column for piece to move to
                if direction.lower() == "down":
                    new_column = piece.col + 1
                    new_row = piece.row + 1
                if direction.lower() == "up":
                    new_column = piece.col - 1
                    new_row = piece.row - 1
                if direction.lower() == "left":
                    new_column = piece.col - 1 
                    new_row = piece.row + 1
                if direction.lower() == "right":
                    new_column = piece.col + 1
                    new_row = piece.row - 1

                # Relocate the piece to the new position without triggering next turn
                piece.board._relocate_no_turn(piece, new_row, new_column)
                
                # Recursively call minimax to get the score
                score = self.minimax(depth + 1, True, alpha, beta, max_depth)

                # Relocate the piece back to its original location without triggering next turn
                piece.board._relocate_no_turn(piece, old_row, old_column)

                # Check if this score is less than the best_score, if so replace the old best_score
                best_score = min(score, best_score)

                # Break if Alpha is >= Beta 
                beta = min(beta, best_score)
                if beta <= alpha:
                    break

            # If there are no legal moves, return 0
            if not legal_moves:
                return 0

            return best_score


    # Move-Getting Minimax Function - Makes the best scored Move             
    def Hounds_Minimax_AI(self, depth, alpha, beta, max_depth=4):

        # Set best score to infinity
        best_score = float("-inf")
        best_move = None
        movers = self.hounds

        # Check legal moves for hounds and put in legal_moves array (moves include [piece, direction])
        legal_moves = [] # (piece, direction)
              
        for piece in self.hounds:
            # candidate directions that preserve dark tiles
            dirs = ["Up", "Left", "Right"]

            for d in dirs:

                dr, dc = self._dir_delta(d)
                if dr is None:
                    continue

                nr, nc = piece.row + dr, piece.col + dc
                if self._in_bounds(nr, nc) and self._is_empty(nr, nc):
                    legal_moves.append((piece, d))
        
        # For each legal move, check if it has the best score             
        for move in legal_moves:

            # Get the piece and direction of the move
            piece, direction = move

            # Save the initial row and column position of the piece to return it to its original location, set up new variables
            old_row = piece.row
            old_column = piece.col
            new_row = 0
            new_column = 0

            # Use directions to create new_row and new_column for piece to move to
            if direction.lower() == "down":
                new_column = piece.col + 1
                new_row = piece.row + 1
            if direction.lower() == "up":
                new_column = piece.col - 1
                new_row = piece.row - 1
            if direction.lower() == "left":
                new_column = piece.col - 1 
                new_row = piece.row + 1
            if direction.lower() == "right":
                new_column = piece.col + 1
                new_row = piece.row - 1

            # Relocate the piece to the new position without triggering next turn
            piece.board._relocate_no_turn(piece, new_row, new_column)

            # Call minimax to get the score
            score = self.minimax(0, False, alpha, beta, max_depth)
    
            # Relocate the piece back to its original location without triggering next turn
            self._relocate_no_turn(piece, old_row, old_column)

            # Check if this score is greater than the best_score, if so replace the old best_score and set the best_move equal to the move that got the score
            if score > best_score:
                best_score = score
                best_move = (piece, direction)

            # Break if Alpha is >= Beta 
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
    
        # Execute the selected best_move
        if best_move:
            piece, direction = best_move
            piece.move(direction)
       
    #helper function to build graph oh game board for dijkstras shortest path algorithm
    def build_graph(self):
        graph = {}
        size = 6  # 6x6 board
        directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]  # Diagonals

        for row in range(size):
            for col in range(size):
                if not self._is_empty(row, col) and not (row, col) == (self.fox.row, self.fox.col): #still count space fox is on
                    continue  # Skip occupied spaces

                neighbors = []

                for dr, dc in directions:
                    new_row, new_col = row + dr, col + dc

                    if (
                        0 <= new_row < size and
                        0 <= new_col < size and
                        self._is_empty(new_row, new_col)
                    ):
                        neighbors.append(((new_row, new_col), 1))  # weight of 1 for adjacent move

                graph[(row, col)] = neighbors

        return graph

    #dijkstras shortest path algorithm
    def dijkstra(self, graph, start):
        pq = [(0, start)]
        distances = {node: float('inf') for node in graph}
        previous = {node: None for node in graph}
        distances[start] = 0

        while pq:
            current_distance, current_node = heapq.heappop(pq)

            if current_distance > distances[current_node]:
                continue

            for neighbor, weight in graph[current_node]:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous[neighbor] = current_node
                    heapq.heappush(pq, (distance, neighbor))

        return distances, previous

    #Get the shortest path to the goal cell 5,5 from current cell
    def reconstruct_path(self, previous, target):
        path = []
        current = target
        while current is not None:
            path.append(current)
            current = previous[current]
        path.reverse()
        return path  # from start to target

    #main logic for shortest path hound AI
    def Fox_Short_Path_AI(self):
        graph = self.build_graph()
        start_node = (self.fox.row, self.fox.col)
        goal_node = (5, 5)

        distances, previous = self.dijkstra(graph, start_node)

        # Direction mapping: (delta_row, delta_col) -> direction name
        delta_to_direction = {
            (-1, -1): "up",
            (1, 1): "down",
            (1, -1): "left",
            (-1, 1): "right"
        }

        def get_direction(from_cell, to_cell):
            dr = to_cell[0] - from_cell[0]
            dc = to_cell[1] - from_cell[1]
            return delta_to_direction.get((dr, dc), None)

        if goal_node in distances and distances[goal_node] != float('inf'):
            # Path exists to goal
            path = self.reconstruct_path(previous, goal_node)
            if len(path) >= 2:
                next_step = path[1]
                direction = get_direction(start_node, next_step)
                if direction:
                    self.fox.move(direction)
        else:
            # No path to (5, 5), move toward closest reachable node to it
            def manhattan(a, b):
                return abs(a[0] - b[0]) + abs(a[1] - b[1])

            best_node = None
            best_score = float('inf')

            for node, dist in distances.items():
                if dist < float('inf'):
                    proximity = manhattan(node, goal_node)
                    if proximity < best_score:
                        best_score = proximity
                        best_node = node
                        
            #If we are already the closest we can be to the goal cell, we cant just do nothing
            #so just move somewhere to pass the turn hoping the hounds will open up a path on
            #a later turn. When this case is hit it essentially means the hounds are blocking
            #any path to the goal.
            if best_node == start_node:
                self.Fox_Random_AI()
                return
            
            if best_node:
                path = self.reconstruct_path(previous, best_node)
                if len(path) >= 2:
                    next_step = path[1]
                    direction = get_direction(start_node, next_step)
                    if direction:
                        self.fox.move(direction)

In [6]:
root = tk.Tk()
root.title("Fox and Hounds")
board = gameboard(root)
board.game_start("ShortestPath", "Player")

In [7]:
# Config
N_GAMES = 10
CONDITIONS = [
    ("FoxRandom",  "HoundsRandom"),     # random-random
    ("ShortestPath","Mini-Max"),        # ai-ai
    ("FoxRandom",  "Mini-Max"),         # random-ai
    ("ShortestPath","HoundsRandom"),    # ai-random

]
MAX_STEPS = 5000 # safety cap for update cycles
SLEEP_SEC = 0.00005 # delay between GUI updates
# SimBoard counts moves
class SimBoard(gameboard):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.turn_count = 0

    def _relocate(self, piece, nr, nc):
        self.turn_count += 1
        return super()._relocate(piece, nr, nc)


def run_single_game(fox_mode, hounds_mode, gnum, max_steps=MAX_STEPS, sleep_sec=SLEEP_SEC):
    root = tk.Tk()
    root.withdraw()
    board = SimBoard(root)

    cond_name = f"{fox_mode.lower()}-{hounds_mode.lower()}".replace("mini-max", "ai")
    print(f"\nGame {gnum:03d} START — Condition: {cond_name}")

    board.game_start(fox_mode, hounds_mode)

    steps = 0
    while not board.game_over and steps < max_steps:
        root.update()
        time.sleep(sleep_sec)
        steps += 1

    if board.game_over:
        winner = board.winner
    else:
        winner = "Timeout"

    turns = getattr(board, "turn_count", None)
    print(f"Game {gnum:03d} END   — Winner: {winner}, Turns: {turns}")

    try:
        root.destroy()
    except Exception:
        pass

    return {"condition": cond_name, "winner": winner, "turns": turns}


def run_batch(n_games=N_GAMES, conditions=CONDITIONS):
    results = []
    gnum = 1
    for fox_mode, hounds_mode in conditions:
        for _ in range(n_games):
            res = run_single_game(fox_mode, hounds_mode, gnum)
            results.append(res)
            gnum += 1
    return pd.DataFrame(results)


# RUN
df_results = run_batch()

# SUMMARY
print("\n" + "="*70)
print("FINAL SUMMARY STATS")
print("="*70)

summary_counts = (
    df_results
    .groupby(["condition","winner"])
    .size()
    .unstack(fill_value=0)
    .sort_index()
)
summary_turns = (
    df_results[df_results["winner"].isin(["Fox","Hounds"])]
    .groupby("condition")["turns"]
    .agg(["count","mean","median","min","max"])
    .sort_index()
)

print("\n-- Win/Loss Counts --")
print(summary_counts)

print("\n-- Turns Summary --")
print(summary_turns)

print("\nDone.")


Game 001 START — Condition: foxrandom-houndsrandom
Game 001 END   — Winner: Hounds, Turns: 42

Game 002 START — Condition: foxrandom-houndsrandom
Game 002 END   — Winner: Hounds, Turns: 52

Game 003 START — Condition: foxrandom-houndsrandom
Game 003 END   — Winner: Fox, Turns: 25

Game 004 START — Condition: foxrandom-houndsrandom
Game 004 END   — Winner: Hounds, Turns: 40

Game 005 START — Condition: foxrandom-houndsrandom
Game 005 END   — Winner: Fox, Turns: 117

Game 006 START — Condition: foxrandom-houndsrandom
Game 006 END   — Winner: Hounds, Turns: 68

Game 007 START — Condition: foxrandom-houndsrandom
Game 007 END   — Winner: Hounds, Turns: 72

Game 008 START — Condition: foxrandom-houndsrandom
Game 008 END   — Winner: Fox, Turns: 57

Game 009 START — Condition: foxrandom-houndsrandom
Game 009 END   — Winner: Hounds, Turns: 42

Game 010 START — Condition: foxrandom-houndsrandom
Game 010 END   — Winner: Fox, Turns: 45

Game 011 START — Condition: shortestpath-ai
Game 011 END   —